In [1]:
#!python3

import os
import pickle
import time

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
def colPreparation():
    labelEncoder = ['Gender','Driving_License','Previously_Insured','Vehicle_Damage']
    oneHotEncoder = ['Vehicle_Age','Region_Code','Policy_Sales_Channel']
    standarScaller = ['Age','Annual_Premium','Vintage']
    
    return labelEncoder, oneHotEncoder, standarScaller

In [3]:
def prepOneHotEncoder(df, col, path):
    filename = 'prep'+col+'.pkl'
    labelOnetHotEncoder = OneHotEncoder()
    dfOneHotEncoder = pd.DataFrame(labelOnetHotEncoder.transform(df[[col]]).toarray(), \
                                   columns = [col+"_"+str(i+1) for i in range(len(df[col].unique()))])
    
    pickle.dump(labelOnetHotEncoder, open(path + filename, 'wb'))
    df = pd.concat([df.drop(col, axis=1), dfOneHotEncoder], axis=1)
    return df

def prepLabelEncoder(df, col, path):
    filename = 'prep'+col+'.pkl'
    labelencoder = LabelEncoder()
    dflabelencoder = pd.DataFrame(labelencoder.transform(df[[col]]), \
                                  columns = [col])
    
    pickle.dump(labelencoder, open(path + filename, 'wb'))
    df = pd.concat([df.drop(col, axis=1), dflabelencoder], axis=1)
    return df

def prepStandarScaler(df, col, path):
    filename = 'prep'+col+'.pkl'
    scallingStandarScaler = StandardScaler()
    dfStandarScaler = pd.DataFrame(scallingStandarScaler.transform(df[[col]]), \
                                   columns = [col])

    pickle.dump(scallingStandarScaler, open(path + filename, 'wb'))
    df = pd.concat([df.drop(col, axis=1), dfStandarScaler], axis=1)
    return df

In [4]:
pathPackages = os.getcwd()+"\\"+"packages"+"\\"
pathData = os.getcwd()+"\\"+"data"+"\\"

In [5]:
data = pd.read_csv(pathData + 'train.csv')
data = data.drop('id', axis=1)
data.head(1)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/agny.nureza/Documents/digital skola/project-7-MLops\\data\\train.csv'

In [ ]:
X = data.drop('Response', axis=1)
y = data['Response']

pickle.dump(X.columns.tolist(), open(pathPackages + 'columnPreparation.pkl','wb'))
colEncoder, colpOneHotEncoder, colStandarScaler = colPreparation()

for col in X.columns:
    if col in colEncoder:
        X = prepLabelEncoder(X, col, pathPackages)
        
    elif col in colpOneHotEncoder:
        X = prepOneHotEncoder(X, col, pathPackages)
        
    elif col in colStandarScaler:
        X = prepStandarScaler(X, col, pathPackages)

pickle.dump(X.columns.tolist(), open(pathPackages + 'columnModelling.pkl','wb'))
X.head(1)

,Gender,Age,Driving_License,Region_Code_1,Region_Code_2,Region_Code_3,Region_Code_4,Region_Code_5,Region_Code_6,Region_Code_7,...,Policy_Sales_Channel_147,Policy_Sales_Channel_148,Policy_Sales_Channel_149,Policy_Sales_Channel_150,Policy_Sales_Channel_151,Policy_Sales_Channel_152,Policy_Sales_Channel_153,Policy_Sales_Channel_154,Policy_Sales_Channel_155,Vintage
0,1,0.333777,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.748795


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

323942 57167 323942 57167


In [ ]:
start = time.time()

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train,y_train)

stop = time.time()
with open(pathPackages + 'model_InsuranceRecommendation.pkl','wb') as file:
    pickle.dump(model, file)
print(f"{stop-start} seconds time training model.....")

0.9986987113952637 seconds time training model.....


In [ ]:
print('total test :', len(y_test))
y_pred = list(model.predict(X_test))
y_true = list(y_test)

total test : 57167


In [ ]:
print(classification_report(y_true, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     49995
           1       0.36      0.19      0.25      7172

    accuracy                           0.86     57167
   macro avg       0.63      0.57      0.58     57167
weighted avg       0.82      0.86      0.84     57167

